In [224]:
import pandas as pd
import re
from datetime import datetime, timedelta
import datetime as dt

In [225]:
earthquakes=pd.read_csv('./VillarricaEarthquakes.csv')

In [226]:
eruptions=pd.read_csv('./VillarricaEruptions.csv')

# Exploring datasets

In [227]:
eruptions.head()

,volcano_number,volcano_name,eruption_number,eruption_category,vei,start_year,start_month,start_day,latitude,longitude,date
0,357120,Villarrica,12184,Confirmed Eruption,1.0,2009.0,11.0,22.0,-39.42,-71.93,2009-11-22
1,357120,Villarrica,12183,Confirmed Eruption,1.0,2009.0,1.0,29.0,-39.42,-71.93,2009-01-29
2,357120,Villarrica,12182,Confirmed Eruption,1.0,2008.0,10.0,26.0,-39.42,-71.93,2008-10-26
3,357120,Villarrica,12181,Confirmed Eruption,1.0,2004.0,8.0,5.0,-39.42,-71.93,2004-08-05
4,357120,Villarrica,12180,Confirmed Eruption,1.0,2003.0,5.0,23.0,-39.42,-71.93,2003-05-23


In [228]:
eruptions=eruptions.rename(columns={'latitude':'volcano_latitude','longitude': 'volcano_longitude'})

In [229]:
eruptions['date']=pd.to_datetime(eruptions['date'],format="%Y/%m/%d")

In [230]:
eruptions['Days since last eruption']=eruptions['date'].diff(periods=-1).dt.days

In [231]:
eruptions.loc[eruptions['eruption_number']==12150,'Days since last eruption']=700

In [232]:
earthquakes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1726 entries, 0 to 1725
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         1726 non-null   object 
 1   mag        1726 non-null   float64
 2   longitude  1726 non-null   float64
 3   latitude   1726 non-null   float64
 4   date       1726 non-null   object 
dtypes: float64(3), object(2)
memory usage: 67.5+ KB


In [233]:
earthquakes['date']=pd.to_datetime(earthquakes['date'],format="%Y/%m/%d")

# Merging datasets

In [234]:
for day in range(0,31):    
    earthquakes[f'date-{day}']=earthquakes['date']-timedelta(days=day)

In [235]:
for x in range(0, 31):
    globals()['villarrica%s' % x] = earthquakes.join(eruptions.set_index('date'), on=f'date-{x}')
    globals()['villarrica%s' % x] = globals()['villarrica%s' % x].dropna()

In [236]:
villarrica=pd.DataFrame()
for i in range(0,31):
    villarrica=pd.concat([villarrica, globals()['villarrica%s' % i]])

In [237]:
villarrica0=earthquakes.join(eruptions.set_index('date'), on='date-0')

In [238]:
villarrica=pd.concat([villarrica,villarrica0])

In [239]:
villarrica=villarrica.drop_duplicates()

# Final adjustments

In [240]:
villarrica

,id,mag,longitude,latitude,date,date-0,date-1,date-2,date-3,date-4,...,volcano_name,eruption_number,eruption_category,vei,start_year,start_month,start_day,volcano_latitude,volcano_longitude,Days since last eruption
747,usp000h4b2,5.9,-74.967,-39.634,2009-11-22,2009-11-22,2009-11-21,2009-11-20,2009-11-19,2009-11-18,...,Villarrica,12184.0,Confirmed Eruption,1.0,2009.0,11.0,22.0,-39.42,-71.93,297.0
1042,usp000d1hj,3.6,-74.294,-38.658,2004-08-05,2004-08-05,2004-08-04,2004-08-03,2004-08-02,2004-08-01,...,Villarrica,12181.0,Confirmed Eruption,1.0,2004.0,8.0,5.0,-39.42,-71.93,440.0
1041,usp000d1mc,2.9,-71.392,-36.886,2004-08-06,2004-08-06,2004-08-05,2004-08-04,2004-08-03,2004-08-02,...,Villarrica,12181.0,Confirmed Eruption,1.0,2004.0,8.0,5.0,-39.42,-71.93,440.0
1611,usp0000mcq,4.8,-73.506,-38.184,1977-01-28,1977-01-28,1977-01-27,1977-01-26,1977-01-25,1977-01-24,...,Villarrica,12166.0,Confirmed Eruption,1.0,1977.0,1.0,26.0,-39.42,-71.93,1916.0
1699,iscgem879719,5.8,-73.555,-38.534,1960-07-05,1960-07-05,1960-07-04,1960-07-03,1960-07-02,1960-07-01,...,Villarrica,12161.0,Uncertain Eruption,1.0,1960.0,7.0,2.0,-39.42,-71.93,604.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1721,iscgem893991,6.2,-73.519,-38.589,1951-01-03,1951-01-03,1951-01-02,1951-01-01,1950-12-31,1950-12-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1722,iscgem896480,6.9,-72.676,-37.665,1949-04-20,1949-04-20,1949-04-19,1949-04-18,1949-04-17,1949-04-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1723,iscgem901645,7.0,-73.630,-42.041,1940-10-11,1940-10-11,1940-10-10,1940-10-09,1940-10-08,1940-10-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1724,iscgem901785,7.8,-72.315,-36.305,1939-01-25,1939-01-25,1939-01-24,1939-01-23,1939-01-22,1939-01-21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [241]:
villarrica['was_erupted']=villarrica['volcano_number'].fillna(0)

In [242]:
villarrica['was_erupted'].loc[villarrica['was_erupted']==357120]=1

C:\Users\Pedro\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [243]:
villarrica['was_erupted'].value_counts()

0.0    1724
1.0      36
Name: was_erupted, dtype: int64

In [244]:
villarrica=villarrica.sort_values(by=['id','eruption_number'])

In [245]:
villarrica.loc[villarrica.duplicated(subset='id',keep='last')]

,id,mag,longitude,latitude,date,date-0,date-1,date-2,date-3,date-4,...,eruption_number,eruption_category,vei,start_year,start_month,start_day,volcano_latitude,volcano_longitude,Days since last eruption,was_erupted
1699,iscgem879719,5.8,-73.555,-38.534,1960-07-05,1960-07-05,1960-07-04,1960-07-03,1960-07-02,1960-07-01,...,12161.0,Uncertain Eruption,1.0,1960.0,7.0,2.0,-39.42,-71.93,604.0,1.0
1611,usp0000mcq,4.8,-73.506,-38.184,1977-01-28,1977-01-28,1977-01-27,1977-01-26,1977-01-25,1977-01-24,...,12166.0,Confirmed Eruption,1.0,1977.0,1.0,26.0,-39.42,-71.93,1916.0,1.0
1610,usp0000mdp,4.7,-74.050,-38.620,1977-01-30,1977-01-30,1977-01-29,1977-01-28,1977-01-27,1977-01-26,...,12166.0,Confirmed Eruption,1.0,1977.0,1.0,26.0,-39.42,-71.93,1916.0,1.0
1609,usp0000mg7,4.5,-73.529,-38.058,1977-02-06,1977-02-06,1977-02-05,1977-02-04,1977-02-03,1977-02-02,...,12166.0,Confirmed Eruption,1.0,1977.0,1.0,26.0,-39.42,-71.93,1916.0,1.0
1552,usp0001ze4,5.1,-74.864,-40.241,1983-10-30,1983-10-30,1983-10-29,1983-10-28,1983-10-27,1983-10-26,...,12168.0,Confirmed Eruption,1.0,1983.0,10.0,14.0,-39.42,-71.93,1211.0,1.0
1433,usp0005et3,5.5,-74.911,-39.035,1992-10-01,1992-10-01,1992-09-30,1992-09-29,1992-09-28,1992-09-27,...,12171.0,Confirmed Eruption,1.0,1992.0,9.0,11.0,-39.42,-71.93,378.0,1.0
1417,usp0006md0,6.0,-70.811,-39.187,1994-10-20,1994-10-20,1994-10-19,1994-10-18,1994-10-17,1994-10-16,...,12172.0,Confirmed Eruption,1.0,1994.0,9.0,26.0,-39.42,-71.93,745.0,1.0
1404,usp0006vx0,4.3,-72.918,-36.361,1995-04-19,1995-04-19,1995-04-18,1995-04-17,1995-04-16,1995-04-15,...,12173.0,Confirmed Eruption,1.0,1995.0,4.0,15.0,-39.42,-71.93,201.0,1.0
1403,usp0006wtv,4.4,-68.220,-37.305,1995-05-04,1995-05-04,1995-05-03,1995-05-02,1995-05-01,1995-04-30,...,12173.0,Confirmed Eruption,1.0,1995.0,4.0,15.0,-39.42,-71.93,201.0,1.0
1402,usp0006xbq,4.4,-74.584,-36.059,1995-05-14,1995-05-14,1995-05-13,1995-05-12,1995-05-11,1995-05-10,...,12173.0,Confirmed Eruption,1.0,1995.0,4.0,15.0,-39.42,-71.93,201.0,1.0


In [246]:
villarrica=villarrica.drop_duplicates(subset='id',keep='first')

In [247]:
villarrica=villarrica.sort_values(by=['date'])

In [248]:
i=0
last_eruption_date=datetime.strptime('1927/01/01', '%Y/%m/%d')
while i<villarrica.shape[0]:
    if villarrica.iloc[i,47]==1:
        villarrica.iloc[i,46]=(villarrica.iloc[i,4]-last_eruption_date)
        last_eruption_date=villarrica.iloc[i,4]
    else:
        villarrica.iloc[i,46]=(villarrica.iloc[i,4]-last_eruption_date)
    i+=1
print(last_eruption_date)

2009-11-22 00:00:00


In [249]:
villarrica['Days since last eruption']=villarrica['Days since last eruption'].apply(lambda x: re.findall('\d*',str(x))[0]).astype(int)

In [250]:
villarrica=villarrica[['id','mag','longitude','latitude','date','Days since last eruption','vei','was_erupted','volcano_latitude','volcano_longitude']]

# Exporting merged dataset

In [251]:
villarrica.to_csv('./villarrica.csv',index=False)